In [ ]:
import sys
sys.path.insert(0,'../')
import rg
from rg.diagrams import diagram, composition_diagram, diagram_set
from rg.interaction import interaction as J 
from rg.interaction import interaction_identity
from rg.interaction import interaction_system
from rg.interaction import composite_interaction as G
from rg.theory import ftheory
from rg.graphs import composite_interaction_graph as fgraph
from rg.propagator import cpropagator, kpropagator

import numpy as np
from sympy import init_printing
init_printing()

# Create some sample primitives

In [ ]:
coagulation = J([[2,1],[0,0]])
coagulation.diagram

In [ ]:
branching = J([[1,2],[0,0]])
branching.diagram

In [ ]:
cross = J([[1,1],[1,1]])
cross.diagram

In [ ]:
branching_alt = J([[1,1],[0,1]])
coagulation_alt = J([[1,1],[1,0]])
coagulation_alt.diagram

# Work with two example graphs

In [ ]:
U = G(branching)*G(cross)*G(coagulation)
composition_diagram(U,compact=True)

In [ ]:
#composition_diagram(T,compact=True)

# The graph theory stuff is in fgraph - e.g. incidence matrix

In [ ]:
GU = fgraph(U,True)
GU

In [ ]:
# I will also start folding new features into this as a top-level object
GU.draw_decomp()

In [ ]:
#GU = fgraph(T,True)#.betti_number()
#GU.gamma_integral

### Convention:

While we could enforce more conventions earlier, here i assume a random edge ordering and now i want to enforce an ordering

I choose to have all external momenta pointing out of the graph (where we connected to the infinity vertext) and start by assuming all flow is towards increasing vid (order of vertices added in construction)

Then i choose the shortest path from the sink vertex (last vertex added) to the source vertex (first added) and i reverse the orientation on the path

We already at this point have some convention- the first row of the inc matrix is the source, the second last is the sink and the last is the infinity vertex used to complete the graph

This would be done for all loops in a general graph but for now we assume one loop

In [ ]:
#choose all edges on the shortest path between source and sink as the "backflow" edges. one of these is the loop momentum
g = fgraph.edges_to_adjaceny_dict(U.edges)
walk = fgraph.shortest_path(g, 0,2)
walk = list(fgraph.expand_path(walk))
g,walk

In [ ]:
#SVG draw arrows on the compact rep, todo

## observe different sign states on the incident matrix 
we start off seeking the standard form, externals point out, internals point "forward" - by default this is not how it looks so we change edge direction in the 2nd matrix

then we invert the shortest path i.e. change the direction of any edge on the shortest path. now we have our standard form that we will generally assume

in this case, in final matrix, column 1 (0-indexed) is the only one that goes from 1 to -1 reading from top to bottom - this is the "backflow edge" in the loop

In [ ]:
inc = GU._inc 
inc

In [ ]:
inc = fgraph.should_toggle_edge_columns(inc) * inc
inc

In [ ]:
inc_inv = fgraph.invert_path(inc, walk)
inc_inv

## Evaluate the momentum equations

this convention above makes the interpretation of the momentum equations less ambiguous. we now keep signs for the momentum as per the incidence matrix.

In [ ]:
GU.edge_system

In [ ]:
#actually from the notes I am not sure about the following:
#1 if the integrals are the same for any external leg structure, then momenta are based only on the number of vertices/propagators - however how do we resolve momenta
#2 there are some trasmutation "vertices" in diags which supposedly have no local residuals and the loops have the same q structure as the 2-nodes - why is that
#3 there are integrals with 3+ vertices that have no dependance on external momenta but that does not add up for me
#fow now im going to do a hack because i cannot justify what we do in the notes; my rule is going to be for the 2-vertex loop I have one rule and for the 3 and above I have another rule
#it seems without some argument, the same way we have casuality differences which must have momenta differences on certain edges

## This leads to integration of graph momenta (consider simple case)...

In [ ]:
# As i have not decided how k will be determined for sure, I leave it as a parameter here
propagators = cpropagator.from_edges(U.edges,k_node_index=1)
[p.display() for p in propagators]

In [ ]:
#this is just the first residue - in this case there is only one anyway
cpropagator.residues(propagators)[0]["value"]

In [ ]:
K = cpropagator.integrate(propagators)
K

## Example with alpha params not needed for linear reduction => integrate on simplex

In [ ]:
propagators = [cpropagator(0,1, -1),cpropagator(1,1,1),cpropagator(0,1, 1)]
[c.display() for c in propagators]

In [ ]:
C = cpropagator.integrate(propagators)
C

In [ ]:
K = kpropagator(C)
K.parametric_integral

In [ ]:
K.reduced_parametric_integral

In [ ]:
K._p

In [ ]:
K.reduced_terms
#add constant prefactor and D prefactor (coefficient of k^2) for book keeping only

In [ ]:
K.used_chung_wu

In [ ]:
K.gamma_integral()

In [ ]:
K.used_chung_wu

In [ ]:
K.gamma_integral(True, elim=["D_1"])
#decompose reduced terms into qm_kernel, qm_constants, d_constants,
#to isolate k^2 we have already factored out D^nu, now this will cancel with D^alpha=D^{d/2-nu} and we will have D^{d/2} and constants^{alpha}

In [ ]:
#composition_diagram(U,True)

behind the scenes we just integrate the kernel directly rather than allowing sympy to integrate quasi mass

In [ ]:
from sympy import integrate,Symbol,simplify
li,power = K.reduced_terms["M"], K.reduced_terms["nu"]
li,power
#what if D1 is in the integral - how do we integrate that sensibly?

In [ ]:
kernel = li.subs(Symbol("D_1"),0)
kernel

In [ ]:
#get the alpha parameter - there is one of them in this case - and integrate over 0,1
a = [ a for a in li.atoms() if not a.is_constant() and a.name[:5]=="alpha"][0]
simplify(integrate(kernel, (a,0,1)))
#after integrating the kernel we take into account the power and prefactors too

# Example with alpha parameters used for linear reduction (work in progress)

In [ ]:
#punch in the example from theor. ref.

In [ ]:
#what if we have something like this - actually before doing this, see if we can avoid this stage because it is essentially taking us
#out of a group where we had a standard propagator basis 
from sympy import symbols,I
k, p, m1, m2, w, D = symbols("k_l, k_p, m_1, m_2, \omega_p, D_0")
P1 =  (-k+p)**2 + (m1 + m2 -I*w )/D
P2 =  k**2+ (-k+p)**2 + (2*m1-I*w)/D
awkprops = list((1/P1,1/P2))
P = awkprops[0] * awkprops[1]
awkprops

In [ ]:
# propagators = [cpropagator(0,1,-1, ["l", "p"]), cpropagator(0,1,1, ["l", "p"], extra_mom_term=Symbol("k")**2) ]
# [p.display() for p in propagators]
# K = cpropagator.integrate(propagators)
# K

In [ ]:
p2 = kpropagator(P)
p2.parametric_integral
#p2.reduced_parametric_integral

#TODO - alpha is gone, bring it back, and determine how to do the integration in each case - ALSO make sure we have a nice normalisation with nothing in numerator
#late rcompare with graph polynomials

In [ ]:
term = list(p2.parametric_integral.as_numer_denom()[1].as_powers_dict().keys())[0]

In [ ]:
from sympy import * 
from rg.propagator import determine_elimination_variable
P = Poly(expand(term), Symbol("k_l"))
P

In [ ]:
#simpler = P.subs(determine_elimination_variable(P), 1)
simpler = P.subs(Symbol("alpha_1"), Symbol("alpha_2")-1)
simpler

In [ ]:
Q = Poly(simpler, Symbol("k_l"))
Q

In [ ]:
A, B = (Q.coeffs()[1]/Q.coeffs()[0]),(Q.coeffs()[2]/Q.coeffs()[0])
A,B

In [ ]:
k = Symbol("k_l")
exps = k**2 +  A*k + B
exps

In [ ]:
Poly(expand(exps.subs(k, k-A/2)), k )

In [ ]:
#I have divided accross and of course it elims but actually i should have squared it in the substituation?
simplify(simpler/Poly(simpler, Symbol("k_l")).all_coeffs()[0])

In [ ]:
#p2.reduced_parametric_integral

In [ ]:
p2.used_chung_wu

In [ ]:
p2._p

In [ ]:
#assert we do not eliminate both params...